In [1]:
%matplotlib inline

In [2]:
import sys
sys.path.insert(0,'..')
import pickle

import visualize
import datasets
import ldle
import numpy as np
# Uncomment to reload a library
from importlib import reload 
visualize = reload(visualize)
ldle = reload(ldle)
datasets = reload(datasets)

matplotlib.get_backend() =  module://ipykernel.pylab.backend_inline
matplotlib.get_backend() =  module://ipykernel.pylab.backend_inline


In [3]:
fpath1 = '../data/ldle/sphere/ldle_obj.dat'
fpath2 = '../../../../codebase/pyLDLE/data/ldle_results_with_code_on_main_branch_6_Jul_1_32_pm/sphere/ldle_obj.dat'
#fpath2 = '../data/ldle_hdsi2/kleinbottle/ldle_obj.dat'

In [4]:
with open(fpath1, "rb") as f:
    X1, labelsMat1, ldle_obj1 = pickle.load(f)
    
with open(fpath2, "rb") as f:
    X2, labelsMat2, ldle_obj2 = pickle.load(f)

In [5]:
def max_abs_diff(a,b):
    return np.max(np.abs(a-b))

In [6]:
def mean_abs_diff(a,b):
    return np.max(np.abs(a-b))

In [7]:
def min_max_mean(a):
    return np.min(a), np.max(a), np.mean(a)

# Check if data is same

In [8]:
max_abs_diff(X1,X2)

0.0

In [9]:
max_abs_diff(labelsMat1,labelsMat2)

0.0

# Check LDLE Obj attributes one by one

In [10]:
max_abs_diff(ldle_obj1.d_e, ldle_obj2.d_e)

0.0

In [11]:
max_abs_diff(ldle_obj1.neigh_dist, ldle_obj2.neigh_dist)

0.0

In [12]:
max_abs_diff(ldle_obj1.neigh_ind, ldle_obj2.neigh_ind)

0

In [13]:
max_abs_diff(ldle_obj1.v0, ldle_obj2.v0)

AttributeError: 'LDLE' object has no attribute 'v0'

In [14]:
max_abs_diff(ldle_obj1.L, ldle_obj2.L)

AttributeError: 'LDLE' object has no attribute 'L'

In [15]:
max_abs_diff(ldle_obj1.lmbda, ldle_obj2.lmbda)

0.0

In [16]:
max_abs_diff(ldle_obj1.phi, ldle_obj2.phi)

0.0

In [17]:
max_abs_diff(ldle_obj1.gamma, ldle_obj2.gamma)

0.0

In [18]:
max_abs_diff(ldle_obj1.Atilde, ldle_obj2.Atilde)

0.7285596496553691

In [37]:
max_abs_diff(ldle_obj1.local_param0.Psi_i[:,0], ldle_obj2.local_param0.Psi_i[:,0])

0

In [38]:
max_abs_diff(ldle_obj1.local_param0.Psi_i[:,1], ldle_obj2.local_param0.Psi_i[:,1])

78

In [39]:
k=np.argmax((ldle_obj1.local_param0.Psi_i[:,1]- ldle_obj2.local_param0.Psi_i[:,1])!=0)

In [43]:
ldle_obj1.local_param0.Psi_i[k,1]

17

In [44]:
ldle_obj2.local_param0.Psi_i[k,1]

43

In [51]:
t_k_ = ldle_obj1.Atilde[k,0,0]/ldle_obj2.Atilde[k,0,0]

In [56]:
from scipy.linalg import inv
def debug_(self, k, t_k=1, tau=50, delta=0.9,d=2):
    # to store i_1, ..., i_d
    i = np.zeros(d, dtype='int')
    # Grab the precomputed U_k, Atilde_{kij}, gamma_{ki}
    U_k = self.U[k,:]
    Atilde_k = self.Atilde[k,:,:]
    gamma_k = self.gamma[k,:]
    print('gamma_k',gamma_k)

    # Compute theta_1
    Atikde_kii = Atilde_k.diagonal()
    print('Atikde_kii',Atikde_kii)
    theta_1 = np.percentile(Atikde_kii, tau)
    print('theta_1',theta_1)

    # Compute Stilde_k
    Stilde_k = Atikde_kii >= theta_1
    print('Stilde_k',Stilde_k)

    # Compute i_1
    r_1 = np.argmax(Stilde_k) # argmax finds first index with max value
    print('r_1',r_1)
    temp = gamma_k * np.abs(Atilde_k[:,r_1])
    print('temp',temp)
    alpha_1 = np.max(temp * Stilde_k)
    print('alpha_1',alpha_1)
    i[0] = np.argmax((temp >= delta*alpha_1) & (Stilde_k))
    print('i0',i[0])
    
    for s in range(1,d):
        i_prev = i[0:s]
        # compute temp variable to help compute Hs_{kij} below
        temp = inv(Atilde_k[np.ix_(i_prev,i_prev)])

        # Compute theta_s
        Hs_kii = Atikde_kii - np.sum(Atilde_k[:,i_prev] * np.dot(temp, Atilde_k[i_prev,:]).T, 1)
        temp_ = Hs_kii[Stilde_k]
        theta_s = np.percentile(temp_, tau)
        print(theta_s, theta_s/t_k)
        theta_s=np.max([theta_s,np.min([np.max(temp_),1e-4])])
        print(theta_s, theta_s/t_k)
        # Compute i_s
        r_s = np.argmax((Hs_kii>=theta_s) & Stilde_k)
        Hs_kir_s = Atilde_k[:,[r_s]] - np.dot(Atilde_k[:,i_prev], np.dot(temp, Atilde_k[i_prev,r_s][:,np.newaxis]))
        temp = gamma_k * np.abs(Hs_kir_s.flatten())
        alpha_s = np.max(temp * Stilde_k)
        i[s]=np.argmax((temp >= delta*alpha_s) & Stilde_k)
        print('i[s]',i[s])

In [57]:
debug_(ldle_obj1, k, t_k_)

gamma_k [6.70144925e+02 6.98572958e+02 5.55269576e+01 4.68792196e+01
 3.15587033e+03 4.21554724e+03 3.13488807e+02 2.59313471e+02
 2.12233236e+02 1.57307254e+02 4.32229980e+01 1.08572964e+03
 1.75069862e+03 1.66098508e+04 1.60393031e+04 1.11289015e+02
 2.24274827e+02 5.79244737e+02 8.53464732e+02 5.28423575e+03
 6.51307015e+03 3.53262960e+01 6.70713827e+04 6.76162875e+04
 3.48072387e+01 3.99493170e+02 6.10810415e+02 2.96879008e+03
 2.56027928e+03 2.66225835e+04 2.46834980e+04 1.57083557e+02
 7.63532182e+01 2.73689858e+05 3.10628201e+05 1.50015034e+03
 1.79502346e+03 4.88454983e+03 1.21706530e+04 3.96266621e+01
 1.10260332e+05 3.10793400e+04 4.14805665e+02 3.11505742e+02
 6.96394378e+01 7.42633061e+01 7.69986814e+05 1.09958950e+06
 3.90605949e+04 5.27422194e+04 5.45980849e+01 8.85934086e+01
 5.77924792e+03 6.16957921e+03 1.13169484e+03 9.86672789e+02
 1.29778059e+05 3.80876560e+05 1.82210472e+02 2.45791339e+02
 3.76723009e+01 2.21002410e+06 9.33354592e+05 2.27474996e+05
 1.87010982e+05 

In [55]:
debug_(ldle_obj2, k)

gamma_k [6.70144925e+02 6.98572958e+02 5.55269576e+01 4.68792196e+01
 3.15587033e+03 4.21554724e+03 3.13488807e+02 2.59313471e+02
 2.12233236e+02 1.57307254e+02 4.32229980e+01 1.08572964e+03
 1.75069862e+03 1.66098508e+04 1.60393031e+04 1.11289015e+02
 2.24274827e+02 5.79244737e+02 8.53464732e+02 5.28423575e+03
 6.51307015e+03 3.53262960e+01 6.70713827e+04 6.76162875e+04
 3.48072387e+01 3.99493170e+02 6.10810415e+02 2.96879008e+03
 2.56027928e+03 2.66225835e+04 2.46834980e+04 1.57083557e+02
 7.63532182e+01 2.73689858e+05 3.10628201e+05 1.50015034e+03
 1.79502346e+03 4.88454983e+03 1.21706530e+04 3.96266621e+01
 1.10260332e+05 3.10793400e+04 4.14805665e+02 3.11505742e+02
 6.96394378e+01 7.42633061e+01 7.69986814e+05 1.09958950e+06
 3.90605949e+04 5.27422194e+04 5.45980849e+01 8.85934086e+01
 5.77924792e+03 6.16957921e+03 1.13169484e+03 9.86672789e+02
 1.29778059e+05 3.80876560e+05 1.82210472e+02 2.45791339e+02
 3.76723009e+01 2.21002410e+06 9.33354592e+05 2.27474996e+05
 1.87010982e+05 

In [ ]:
max_abs_diff(ldle_obj1.local_param0.zeta, ldle_obj2.local_param0.zeta)

In [ ]:
min_max_mean(ldle_obj1.local_param0.zeta)

In [ ]:
min_max_mean(ldle_obj2.local_param0.zeta)

In [ ]:
max_abs_diff(ldle_obj1.local_param0.Psi_gamma, ldle_obj2.local_param0.Psi_gamma)

In [ ]:
max_abs_diff(ldle_obj1.local_param.Psi_i, ldle_obj2.local_param.Psi_i)

In [ ]:
min_max_mean(ldle_obj1.local_param.zeta)

In [ ]:
min_max_mean(ldle_obj2.local_param.zeta)

In [ ]:
max_abs_diff(ldle_obj1.local_param.Psi_gamma, ldle_obj2.local_param.Psi_gamma)